# Simple Classification Model
# Author : Rohit Kumar Hansdah 
# Last Updated : 26 December 2020
# Refrences : 
              
              
1. [https://www.kaggle.com/beletecheneke/xgboost-with-roc-curve-notebook-auprc-93](http://)              
2. [https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html](http://)

3. [https://www.kaggle.com/saurabhshahane/random-forest-and-xgboost-for-beginners](http://)

4. [https://www.kaggle.com/andleebhayath/lgb-catboost-random-forest-and-xgboost](http://)

# Importing Libraries

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc,recall_score,precision_score
from sklearn.metrics import roc_auc_score

# Importing Dataset

In [ ]:
full_train = pd.read_csv("/kaggle/input/riiid-test-answer-prediction/train.csv",nrows=1000000,low_memory=False)
test_df=pd.read_csv('../input/riiid-test-answer-prediction/example_test.csv')
questions = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv')
lectures = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/lectures.csv')

# Exploratory Data Analysis

# Working on Test Dataset

# Handling Missing Data

In [ ]:
full_train = full_train.fillna(full_train.mean())
test_df = test_df.fillna(test_df.mean())

In [ ]:
full_train.head()

# Working on Question Dataset

# Handling Missing Data

In [ ]:
questions = questions.fillna(questions.mean())

In [ ]:
questions.head()

# Most Useful Question Tags

In [ ]:
import plotly.express as px
check = questions['tags'].str.split(' ').explode('tags').reset_index()
check = check['tags'].value_counts().reset_index()

check.columns = [
    'question_tags', 
    'count'
]

check['question_tags'] = check['question_tags'].astype(str) + '-'
check = check.sort_values(['count']).tail(5)

fig = px.bar(
    check, 
    x='count', 
    y='question_tags', 
    orientation='h', 
     title='Top 5 most useful tags', 
    width=500,
    height=500 
)
fig.show()

In [ ]:
check=full_train[['user_id','content_id','content_type_id','prior_question_had_explanation']]
bundle_id=[]
correct_answer=[]
part=[]
prior_question_had_explanation=[]
question_tags=[]
for i in range(0,check.shape[0]):
    if(check['content_type_id'][i]==0):
        bundle_id.append(questions['bundle_id'][check['content_id'][i]])
        correct_answer.append(questions['correct_answer'][check['content_id'][i]])
        part.append(questions['part'][check['content_id'][i]])
        if(check['prior_question_had_explanation'][i]==True):
            prior_question_had_explanation.append(1)
        else:
            prior_question_had_explanation.append(0)
        s=questions['tags'][check['content_id'][i]]
        temp=""
        score=0
        for j in range(0,len(s)):
            if s[j]==" " or j==len(s)-1:
                if j==len(s)-1:
                    temp+=s[j]
                if temp=="92":
                    score+=5
                elif temp=="38":
                    score+=4
                elif temp=="81":
                    score+=3
                elif temp=="29":
                    score+=2
                elif temp=="136":
                    score+=1
                else:
                    score+=0
                temp=""
            else:
                temp+=s[j]
        question_tags.append(score)
    else:
        bundle_id.append(-1)
        correct_answer.append(-1)
        part.append(-1)
        prior_question_had_explanation.append(-1)
        question_tags.append(-1)
full_train['bundle_id']=bundle_id
full_train['correct_answer']=correct_answer
full_train['part']=part
full_train['prior_question_had_explanation']=prior_question_had_explanation
full_train['question_tags']=question_tags

In [ ]:
c=test_df[['user_id','content_id','content_type_id','prior_question_had_explanation']]
bundle_id=[]
correct_answer=[]
part=[]
prior_question_had_explanation=[]
question_tags=[]
for i in range(0,c.shape[0]):
    if(c['content_type_id'][i]==0):
        bundle_id.append(questions['bundle_id'][c['content_id'][i]])
        correct_answer.append(questions['correct_answer'][c['content_id'][i]])
        part.append(questions['part'][c['content_id'][i]])
        if(c['prior_question_had_explanation'][i]==True):
            prior_question_had_explanation.append(1)
        else:
            prior_question_had_explanation.append(0)
        s=questions['tags'][check['content_id'][i]]
        temp=""
        score=0
        for j in range(0,len(s)):
            if s[j]==" " or j==len(s)-1:
                if j==len(s)-1:
                    temp+=s[j]
                if temp=="92":
                    score+=5
                elif temp=="38":
                    score+=4
                elif temp=="81":
                    score+=3
                elif temp=="29":
                    score+=2
                elif temp=="136":
                    score+=1
                else:
                    score+=0
                temp=""
            else:
                temp+=s[j]
        question_tags.append(score)
    else:
        bundle_id.append(-1)
        correct_answer.append(-1)
        part.append(-1)
        prior_question_had_explanation.append(-1)
        question_tags.append(-1)
test_df['bundle_id']=bundle_id
test_df['correct_answer']=correct_answer
test_df['part']=part
test_df['prior_question_had_explanation']=prior_question_had_explanation
test_df['question_tags']=question_tags

In [ ]:
full_train = full_train[['timestamp','user_id','content_id','content_type_id','task_container_id','user_answer','prior_question_elapsed_time','prior_question_had_explanation','bundle_id','correct_answer','part','prior_question_had_explanation','question_tags','answered_correctly']]
full_train.head()

# Correlation Heatmap

In [ ]:
sns.heatmap(full_train.corr(), 
        xticklabels=full_train.corr().columns,
        yticklabels=full_train.corr().columns)

# Correlation matrix

In [ ]:
full_train.corr()

In [ ]:
X = full_train.iloc[:, :-1].values
y = full_train.iloc[:, -1].values

# Splitting the dataset into train and test 

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

# LGBM and Bagging Classifier

In [ ]:
from sklearn import model_selection 
from sklearn.ensemble import BaggingClassifier 
import lightgbm as lgb
kfold = model_selection.KFold(n_splits = 3)
# initialize the base classifier 
base_clf = lgb.LGBMClassifier()
# no. of base classifier 
num_trees = 100
# bagging classifier 
model = BaggingClassifier(base_estimator = base_clf, n_estimators = num_trees)
model.fit(X_train, y_train)
model_pred=model.predict_proba(X_test)

In [ ]:
from sklearn.metrics import roc_auc_score
print('ROC-AUC Score of LGBM Classifier :')
roc_auc_score(y_test, model_pred, multi_class="ovo")

# Model Predictions on Train Set

In [ ]:
model.predict(X_test)

# Working on Test Dataset

# Creating Environment

In [ ]:
import riiideducation
env = riiideducation.make_env()
iter_test = env.iter_test()

In [ ]:
test_df = test_df[['timestamp','user_id','content_id','content_type_id','task_container_id','user_answer','prior_question_elapsed_time','prior_question_had_explanation','bundle_id','correct_answer','part','prior_question_had_explanation','question_tags','answered_correctly']]
test_df.head()

# Submission File

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    features=['timestamp','user_id','content_id','content_type_id','task_container_id','user_answer','prior_question_elapsed_time','prior_question_had_explanation','bundle_id','correct_answer','part','prior_question_had_explanation','question_tags']
    test_df['answered_correctly'] = model.predict(test_df[features])
    cols_to_submission = ['row_id', 'answered_correctly', 'group_num']
    env.predict(test_df.loc[test_df['content_type_id'] == 0, [cols_to_submission]])

# Code Completed  ^_^